# Import Libraries



In [ ]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd

### IGNORED, DISMISSED: How the NYPD Neglects 311 Complaints about Driver Misconduct

1. Go to the New Data Portal: https://opendata.cityofnewyork.us/
2. Find each dataset. 311 requests and police precinct
3. Add a data notes section to your github readme. This should include information about the source of your data, where it came from, how often it is updated, and the period of data it contains (e.g. 2010-present or 2012 to 2019)
4. Lastly, test that this import works. This may take a while as the dataset is large.

In [ ]:
# query NYC 311 for driver misconduct
# value list
complaint_types_dm = ('Blocked Bike Lane', 'Blocked Crosswalk', 'Blocked Hydrant', 'Blocked Sidewalk', 'Chronic Speeding', 'Chronic Stoplight Violation','Detached Trailer', 'Double Parked Blocking Traffic',
       'Double Parked Blocking Vehicle', 'Drag Racing',
       'Overnight Commercial Storage', 'Parking Permit Improper Use',
       'Posted Parking Sign Violation', 'Truck Route Violation',
       'Unauthorized Bus Layover', 'Abandoned Vehicle With License Plate' )
query = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=" +  requests.utils.quote(f"SELECT * WHERE descriptor IN {complaint_types_dm}  LIMIT 3000000")
dm  = pd.read_csv(query)

<ipython-input-225-c56bcde3c9d2>:9: DtypeWarning: Columns (15,17,18,20,31,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dm  = pd.read_csv(query)


In [ ]:
# get police precint data

police = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON')